In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
train = pd.read_csv('train.tsv', sep='\t')
train.head()

In [ ]:
Y = train['rel']
X = train[train.columns.delete((0, 1, 2))]
X.shape

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42, stratify=Y)

In [ ]:
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Activation, Lambda
from keras.optimizers import Adam

x = Input(shape=(X.shape[1],))

ranking_nn = Sequential()
ranking_nn.add(Dense(23))
ranking_nn.add(Activation('sigmoid'))
ranking_nn.add(Dense(10))
ranking_nn.add(Activation('sigmoid'))
ranking_nn.add(Dense(5))
ranking_nn.add(Activation('sigmoid'))
ranking_nn.add(Dense(1))

rank = ranking_nn(x)

model = Model(inputs=x, outputs=rank)
model.compile(loss='mse',
              optimizer=Adam(lr=5e-6))

In [ ]:
model.fit(X_train, Y_train,
          epochs=10,
          batch_size=8,
          shuffle=True,
          validation_data=(X_test, Y_test),
          verbose=1)
model.fit(X_train, Y_train,
          epochs=10,
          batch_size=16,
          shuffle=True,
          validation_data=(X_test, Y_test),
          verbose=1)
model.fit(X_train, Y_train,
          epochs=10,
          batch_size=32,
          shuffle=True,
          validation_data=(X_test, Y_test),
          verbose=1)

In [ ]:
test = pd.read_csv('test.tsv', sep='\t')
test.head()

In [ ]:
res = test[['docid', 'quid', 'feature_0']].copy()
res['feature_0'] = model.predict(test[test.columns.delete((0, 1))])
res.head()

In [ ]:
csv = res.sort_values(['quid', 'feature_0'], ascending=False)[['docid', 'quid']]
csv.to_csv('submission.csv', index=False)